In [1]:
from ngsolve import *
from netgen.geom2d import *

import numpy as np
import scipy.linalg
import scipy.sparse as sp

from netgen.occ import *

from helping_functions import *

from ngsolve.webgui import Draw
import matplotlib.pyplot as plt


In [2]:
# DOMAIN CONSTRUCTION Omega = B1(0)
# Division in 8 subdomains

geo = SplineGeometry()
Points = [(0,-1), (1,-1), (1,0), 
          (1,1), (0,1), (-1,1),
          (-1,0), (-1,-1), (0,0)]

bcs_edge = ["c0", "c1", "c2", "c3", 
            "m0", "m1", "m2", "m3",
            "m4", "m5", "m6", "m7"]

for i, pnt in enumerate(Points):
    geo.AddPoint(*pnt, name = "V" + str(i))

geo.Append(["spline3", 0, 1, 2], leftdomain=1, rightdomain=0, bc="c0")
geo.Append(["spline3", 2, 3, 4], leftdomain=2, rightdomain=0, bc="c1")
geo.Append(["spline3", 4, 5, 6], leftdomain=3, rightdomain=0, bc="c2")
geo.Append(["spline3", 6, 7, 0], leftdomain=4, rightdomain=0, bc="c3")
geo.Append(["line", 0, 2], leftdomain=5, rightdomain=1, bc="m0")
geo.Append(["line", 2, 4], leftdomain=6, rightdomain=2, bc="m1")
geo.Append(["line", 4, 6], leftdomain=7, rightdomain=3, bc="m2")
geo.Append(["line", 6, 0], leftdomain=8, rightdomain=4, bc="m3")

geo.Append(["line", 8, 0], leftdomain=5, rightdomain=8, bc="m4")
geo.Append(["line", 8, 2], leftdomain=6, rightdomain=5, bc="m5")
geo.Append(["line", 8, 4], leftdomain=7, rightdomain=6, bc="m6")
geo.Append(["line", 8, 6], leftdomain=8, rightdomain=7, bc="m7")

# geo = SplineGeometry()
# geo.AddCircle ( (0, 0), r=1, leftdomain=1, rightdomain=0, )
ngmesh = geo.GenerateMesh(maxh= 0.02) #Too much 0.0013




mesh = Mesh(ngmesh)
for i in range(8):
    mesh.ngmesh.SetMaterial(i+1,"om" + str(i))

#Draw(mesh)
print(mesh.nv)
# quit()
print(mesh.GetMaterials())
print(mesh.GetBoundaries())
print(mesh.GetBBoundaries())
# input()

dom_bnd = "c0|c1|c2|c3"



9070
('om0', 'om1', 'om2', 'om3', 'om4', 'om5', 'om6', 'om7')
('c0', 'c1', 'c2', 'c3', 'm0', 'm1', 'm2', 'm3', 'm4', 'm5', 'm6', 'm7')
('V0', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8')


In [3]:
#Draw(mesh)

**Helmholtz problem description - Plane wave**

$- div (a \nabla u) - k^2 u  = f$ in $\Omega = B_1(0)$,

$ a \partial_n u - i \omega \beta u = g $ on $\partial \Omega = \Gamma_R$. 


In this example we have:

$a=1$, 

$c=1$, 

$\beta=1$, 

$\omega=2$, 

$k = \omega (0.6,0.8)$,

$f=0$,

$g= -i \left((0.6 x + 0.8 y) + \beta \right) \exp^{-i (0.6 x + 0.8 y)}$

$u_{ex} = \exp^{-i (0.6 x + 0.8 y)}$.

In [4]:
#Problem setting - PLANE WAVE

omega = 1
#CF = CoefficientFunction
kappa = omega
k = kappa * CF((0.6,0.8))
beta = 1
f = 0
u_ex = exp(-1J * (k[0] * x + k[1] * y))
g = -1j * kappa * (k[0] * x + k[1] * y) * u_ex - 1j *beta * u_ex

#Draw(g.real, mesh, "u_ex")
#Draw(g.imag, mesh, "u_ex")


**Helmholtz problem description - Interior source**

$- div (a \nabla u) - k^2 u  = f$ in $\Omega = B_1(0)$,

$ a \partial_n u - i \omega \beta u = g $ on $\partial \Omega = \Gamma_R$. 


In this example we have:

$a=1$, 

$c=1$, 

$\beta=1$, 

$\omega=1$, 

$k = \omega$,

$f=\exp^{(-200 |x-x_c|^2)}$, with $x_c = (1/3,1/3)$,

$g= 0$.

In [ ]:
class Point:
    """ Point class for representing and manipulating x,y coordinates. """

    def __init__(self):
        """ Create a new point at the origin """
        self.x = 0
        self.y = 0

In [ ]:
#Problem setting - INTERIOR SOURCE

omega = 1
kappa = omega
beta = 1
p = (1/3, 1/3)
P = Point()
P.x = 1/3
P.y = 1/3

f = exp(-200 * ((x-P.x)**2 + (y-P.y)**2)) 
g = 0


In [5]:
# Coefficients

order_v = [1] # The following error is due to low resolution -> refine mesh
             # Cannot use scipy.linalg.eig for sparse A with k >= N - 1.
    
Bubble_modes = [1]
Edge_modes = [1,2,4,8]#,32,64,128,256] #[24; 28; : : : ; 3072] #We have 12 edges
max_bm = Bubble_modes[-1]
max_em = Edge_modes[-1]


In [6]:
h1_error = []
dofs =[]


for order in order_v:
    print(order)
    V = H1(mesh, order = order, complex = True)
    u, v = V.TnT()


    a = BilinearForm(V)
    a += grad(u) * grad(v) * dx() 
    a += - kappa**2 * u * v * dx()  
    a += -1J * omega * beta * u * v * ds(dom_bnd)
    a.Assemble()

    l = LinearForm(V)
    l += f * v * dx(bonus_intorder=10)
    l += g * v * ds(dom_bnd,bonus_intorder=10)
    l.Assemble()

    gfu_ex = GridFunction(V)
    ainv = a.mat.Inverse(V.FreeDofs(), inverse = "sparsecholesky")
    gfu_ex.vec.data = ainv * l.vec
    print("FEM finished")
    
    
    gfu = GridFunction(V)
    #Computing full basis with max number of modes 
    acms = ACMS(order = order, mesh = mesh, bm = max_bm, em = max_em)
    print(order)
    
    acms.CalcHarmonicExtensions(kappa = kappa)
    acms.calc_basis()

#     Draw(gfu_ex, mesh, "u_fem")
    

    for EM in Edge_modes:
            for BM in Bubble_modes:
                #Vc = H1(mesh, order = order, complex = True)
                gfu = GridFunction(V)
                basis = MultiVector(gfu.vec, 0)
                
                for bv in acms.basis_v:
                    gfu.vec[:] = 0
                    gfu.vec.FV()[:] = bv
                    basis.Append(gfu.vec)
                
                for e, label in enumerate(mesh.GetBoundaries()):
                    for i in range(EM):
                        #gfu.vec[:] = 0
                        gfu.vec.FV()[:] = acms.basis_e[e * max_em + i]
                        basis.Append(gfu.vec)
        
                for d, dom in enumerate(mesh.GetMaterials()):
                    for i in range(BM):
                        gfu.vec.FV()[:] = acms.basis_b[d * max_bm + i]
                        basis.Append(gfu.vec)

    
                num = len(basis)
                dofs.append(num)
            
                asmall = np.zeros((num, num), dtype=np.complex128)
                asmall = InnerProduct (basis, a.mat * basis, conjugate = False) #Complex
                
                      
                asmall_np = np.zeros((num, num), dtype=np.complex128)
                asmall_np = asmall.NumPy()
    
                ainvs_small_np = np.zeros((num, num), dtype=np.complex128)
                ainvs_small_np = np.linalg.inv(asmall_np)

                ainvsmall = np.zeros((num, num), dtype=np.complex128)
                ainvsmall = Matrix(num,num, complex=True)


                for i in range(num):
                    for j in range(num):
                        ainvsmall[i,j] = ainvs_small_np[i,j]
                 
                f_small = []
                usmall = []
                
                f_small = InnerProduct(basis, l.vec, conjugate=False)
                
                usmall = ainvsmall * f_small
                
                gfu.vec[:] = 0.0
                gfu.vec.data = basis * usmall

                
#                 Draw(gfu-gfu_ex, mesh, "error")

                print("finished_acms")
    
                #Computing ERROR
        
                grad_uex = Grad(gfu_ex)
                diff = grad_uex - Grad(gfu)
                
                h1_error_aux = sqrt( Integrate ( InnerProduct(diff,diff), mesh, order = 15))
                #print(h1_error_aux)
                
                #Needs to do complex conjugate
                #Draw(gfu, mesh, "u_acms")
                h1_error.append(h1_error_aux.real)
            
            

h1_error = np.reshape(h1_error, (len(order_v)*len(Edge_modes), len(Bubble_modes)))
dofs = np.reshape(dofs, (len(order_v)*len(Edge_modes), len(Bubble_modes)))
# print(h1_error)



1
FEM finished
1
finished_acms
finished_acms
finished_acms
finished_acms


In [7]:
h1_tab = h1_error.T
print(h1_tab)

[[0.13568968 0.05853565 0.01662826 0.00446872]]


First test for **classical Helmholtz - Tables 5.3 and 5.5**.

Remark. Tables 5.2 and 5.4 can only be reproduced if we compare with the explicit solution and not the reconstructed FEM.


Test carried out with:<br>
Bubble_modes = [2]   (not needed because there is no source term) <br>
Edge_modes = [2,4,8,16,32]<br>
If the results are compared to the paper, Table 5.2 - 5.5 (second half for $H^1$ error),  <br>
<br>


Something is wrong - There is no convergence!!!<br>
**Table 5.3 - k=1** <br>
**9070** vertices  -  maxh=0.02<br>
[[0.19638151 0.21907383 0.23493477 0.2408995  0.24121711]]<br>


36404 vertices<br>
<br>

First test for **localised interior source - Table 5.6**.

Test carried out with:<br>
Bubble_modes = [2,4,8,16,32]<br>
Edge_modes = [2,4,8,16,32]<br>
If the results are compared to the paper, Table 5.6 (second half for $H^1$ error), it seems there is a shift of $10^{-2}$ in the results. **WHY?** <br>
The behaviour is the same:<br>
-when increasing the number of bubbles, there is a decrease by a factor of 10 at first, then by a factor of (1.2-1.4) before being halved;<br>
-when increasing the number of bubble modes, and almost no convergence when increasing the number of edge modes. <br>


**9070** vertices   -   maxh= 0.02   <br>
[[0.00442196 0.00424678 0.00423678 0.0042367  0.00423669]<br>
 [0.00328148 0.0030442  0.00303053 0.00303041 0.0030304 ]<br>
 [0.00264233 0.00233963 0.00232186 0.00232171 0.00232171]<br>
 [0.00196496 0.00153383 0.00150645 0.00150621 0.00150621]<br>
 [0.00142522 0.00072262 0.0006625  0.00066196 0.00066195]]<br>


Test carried out with:<br>
Bubble_modes = [2,4,8,16,32,64,128]<br>
Edge_modes = [2,4,8,16,32,64]<br>

**36404** vertices  -   maxh= 0.01 <br>
 [[4.43480410e-03 4.26011474e-03 4.25014758e-03 4.25006290e-03  4.25006089e-03 4.25006076e-03]<br>
 [3.29424319e-03 3.05791780e-03 3.04429565e-03 3.04417748e-03  3.04417466e-03 3.04417447e-03]<br>
 [2.65429756e-03 2.35307169e-03 2.33540290e-03 2.33524892e-03  2.33524525e-03 2.33524501e-03]<br>
 [1.97422424e-03 1.54558483e-03 1.51840794e-03 1.51817559e-03  1.51816993e-03 1.51816956e-03]<br>
 [1.42950812e-03 7.30846476e-04 6.71437969e-04 6.70904173e-04  6.70891381e-04 6.70890539e-04]<br>
 [1.27328091e-03 3.34645982e-04 1.69477230e-04 1.67358749e-04  1.67307485e-04 1.67304109e-04]<br>
 [1.26234614e-03 2.90246963e-04 3.12063951e-05 1.61222779e-05  1.55808742e-05 1.55445727e-05]]<br>
 
 
 **WHY** there is a slight increase in the error when we refine the mesh? 

In [ ]:
## PLOT ERROR with increased EDGE modes for different DEGREES of approximation
plt.rcParams.update({'font.size':15})
for d in range(len(order_v)):
    for i in range(len(Bubble_modes)):
        plt.plot(Edge_modes, h1_error[d*len(Edge_modes) : d*len(Edge_modes) + len(Edge_modes),i], label=('$I_b$=%i, p=%i' %(Bubble_modes[i],order_v[d])))
plt.title('$H^1$ errors: increased edge modes')
plt.legend()
plt.xlabel('Edge modes')

In [ ]:
## PLOT ERROR with increased BUBBLE modes for different DEGREES of approximation
plt.rcParams.update({'font.size':15})
for d in range(len(order_v)):
    for i in range(len(Edge_modes)):
        plt.plot(Bubble_modes, h1_error[d*len(Edge_modes) + i,:], label=('$I_e$=%i, p=%i' %(Edge_modes[i],order_v[d])))
plt.title('$H^1$ errors: increased bubbles')
plt.legend()
plt.xlabel('Bubbles')

Old code for bubble and edge modes plots

In [ ]:
#Bubbles
plt.rcParams.update({'font.size':10})
for d in range(len(order_v)):
    for i in range(len(Edge_modes)):
        plt.loglog(Bubble_modes, h1_error[d*len(Edge_modes) + i,:], label=('Edge modes=%i' %Edge_modes[i]))
plt.title('$H^1$ errors: increased bubbles deg=%i' %order)
plt.legend()
plt.xlabel('Bubbles')



In [ ]:
#Edges
plt.rcParams.update({'font.size':10})
for d in range(len(order_v)):
    for i in range(len(Bubble_modes)):
        plt.loglog(dofs[d*len(Edge_modes):(d+1)*len(Edge_modes),i], h1_error[d*len(Edge_modes):(d+1)*len(Edge_modes),i], label=('Bubbles=%i' %Bubble_modes[i]))
plt.title('$H^1$ errors: increased edge modes deg=%i' %order)
plt.legend()
plt.xlabel('Edge modes / Dofs')
